In [2]:
from pyspark.sql.functions import col

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("TrackTimePrediction") \
    .getOrCreate()

spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/11 13:54:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
file_path1 = "./tfrrs_dataall.csv"
file_path2 = "./tfrrs_remaining_archives.csv"
file_path3 = "./tfrrs_current.csv"

df1 = spark.read.csv(file_path1, header=True, inferSchema=True)
df2 = spark.read.csv(file_path2, header=True, inferSchema=True)
df3 = spark.read.csv(file_path3, header=True, inferSchema=True)

combined_df = df1.unionByName(df2).unionByName(df3)

combined_df.printSchema()

combined_df.show(10)

root
 |-- _c0: integer (nullable = true)
 |-- athlete_link: string (nullable = true)
 |-- athlete_name: string (nullable = true)
 |-- meet_date: string (nullable = true)
 |-- rank: integer (nullable = true)
 |-- team_link: string (nullable = true)
 |-- team_name: string (nullable = true)
 |-- time: string (nullable = true)
 |-- year: string (nullable = true)

+---+--------------------+----------------+------------+----+--------------------+--------------+--------+----+
|_c0|        athlete_link|    athlete_name|   meet_date|rank|           team_link|     team_name|    time|year|
+---+--------------------+----------------+------------+----+--------------------+--------------+--------+----+
|  0|https://tf.tfrrs....|   Wolfe, Parker| May 9, 2024|   1|https://tf.tfrrs....|North Carolina|13:19.38|JR-3|
|  1|https://tf.tfrrs....|Mullarkey, David|Apr 26, 2024|   2|https://tf.tfrrs....| Florida State|13:28.43|JR-3|
|  2|https://tf.tfrrs....|    Martin, Gary|Mar 28, 2024|   3|https://tf.tfrrs.

25/06/11 14:01:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , athlete_link, athlete_name, meet_date, rank, team_link, team_name, time, year
 Schema: _c0, athlete_link, athlete_name, meet_date, rank, team_link, team_name, time, year
Expected: _c0 but found: 
CSV file: file:///Users/emccauley6/Documents/GitHub/prism/tfrrs_dataall.csv


In [4]:
output_path = "./tfrrs_database"
combined_df.coalesce(1).write \
    .option("header", "true") \
    .mode("overwrite") \
    .csv(output_path)

25/06/11 14:06:38 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , athlete_link, athlete_name, meet_date, rank, team_link, team_name, time, year
 Schema: _c0, athlete_link, athlete_name, meet_date, rank, team_link, team_name, time, year
Expected: _c0 but found: 
CSV file: file:///Users/emccauley6/Documents/GitHub/prism/tfrrs_dataall.csv
25/06/11 14:06:38 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , athlete_link, athlete_name, meet_date, rank, team_link, team_name, time, year
 Schema: _c0, athlete_link, athlete_name, meet_date, rank, team_link, team_name, time, year
Expected: _c0 but found: 
CSV file: file:///Users/emccauley6/Documents/GitHub/prism/tfrrs_remaining_archives.csv
25/06/11 14:06:38 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , athlete_link, athlete_name, meet_date, rank, team_link, team_name, time, year
 Schema: _c0, athlete_link, athlete_name, meet_date, rank, team_link, team_name, t

In [5]:
spark.stop()